In [4]:
import pickle

pickle_file_path = "/ceph/cms/store/user/dprimosc/bbbb/skimmer/24Sep3_v12_private_signal/2022EE/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV/pickles/out_0.pkl"

with open(pickle_file_path, "rb") as f:
    data = pickle.load(f)

In [5]:
print(type(data))

print("Keys in the dictionary:")
for key in data.keys():
    print(key)
    print(data["2022EE"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV"])

<class 'dict'>
Keys in the dictionary:
2022EE
{'totals': {'nevents': 12906.811, 'np_nominal': 13594.458669231468, 'np_pileupUp': 15657.315861215926, 'np_FSRPartonShowerUp': 13588.178756881793, 'np_ISRPartonShowerDown': 13592.752311372888, 'np_pileupDown': 12474.824248229534, 'np_ISRPartonShowerUp': 13595.824607034214, 'np_FSRPartonShowerDown': 13590.163419752673, 'np_scale_weights': array([15486.12385103, 15244.69304213, 13893.63777244, 13361.73234113,
       12138.2039158 , 11866.18527195]), 'np_pdf_weights': array([13594.04397341, 13592.45863184, 13565.0643261 , 13598.69707564,
       13627.21034384, 13586.25735023, 13628.49238302, 13615.98294382,
       13629.41448917, 13584.79352947, 13586.78209017, 13572.66650775,
       13618.99608119, 13605.74381279, 13621.86932921, 13617.27292378,
       13600.5270745 , 13601.48363351, 13584.30111886, 13590.25375501,
       13580.98766202, 13586.18167038, 13582.1539826 , 13605.7208795 ,
       13565.74535264, 13592.4431713 , 13579.40365327, 135

In [7]:
signal = data["2022EE"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV"]

for key in signal.keys():
    print(key)

signal["totals"]["nevents"]
signal["cutflow"]

totals
cutflow


OrderedDict([('all', 12906.811),
             ('has_bbbb', 12906.811),
             ('trigger', 807.4955),
             ('met_filters', 806.53503),
             ('ak4_jetveto', 726.58075),
             ('ak8_numjets', 271.72943),
             ('0lep', 266.5474)])

In [8]:
signal["totals"]

{'nevents': 12906.811,
 'np_nominal': 13594.458669231468,
 'np_pileupUp': 15657.315861215926,
 'np_FSRPartonShowerUp': 13588.178756881793,
 'np_ISRPartonShowerDown': 13592.752311372888,
 'np_pileupDown': 12474.824248229534,
 'np_ISRPartonShowerUp': 13595.824607034214,
 'np_FSRPartonShowerDown': 13590.163419752673,
 'np_scale_weights': array([15486.12385103, 15244.69304213, 13893.63777244, 13361.73234113,
        12138.2039158 , 11866.18527195]),
 'np_pdf_weights': array([13594.04397341, 13592.45863184, 13565.0643261 , 13598.69707564,
        13627.21034384, 13586.25735023, 13628.49238302, 13615.98294382,
        13629.41448917, 13584.79352947, 13586.78209017, 13572.66650775,
        13618.99608119, 13605.74381279, 13621.86932921, 13617.27292378,
        13600.5270745 , 13601.48363351, 13584.30111886, 13590.25375501,
        13580.98766202, 13586.18167038, 13582.1539826 , 13605.7208795 ,
        13565.74535264, 13592.4431713 , 13579.40365327, 13575.16478387,
        13596.03000428, 1354